In [53]:
print "IMPORTING LIBRARIES..."
import pandas as pd
import numpy as np
import statsmodels.api as sm
import re
import requests
from requests.auth import HTTPBasicAuth


#DOWLOADING FILE FROM DROPBOX FIRST TIME
import urllib2
import os.path
import time
import random
while not os.path.exists('dev.csv') or not os.path.exists('oot0.csv'):
    time.sleep (3*random.random()); #Sleeping less than 3 seconds before going to Dropbox - avoid too many students at once.
    if not os.path.exists('dev.csv'):
        print "DOWLOADING FILE dev.csv FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!"
        csvfile = urllib2.urlopen("https://dl.dropboxusercontent.com/u/28535341/dev.csv")
        output = open('dev.csv','wb')
        output.write(csvfile.read())
        output.close()
    if not os.path.exists('oot0.csv'):
        print "DOWLOADING FILE oot0.csv FROM DROPBOX BECAUSE LOCAL FILE DOES NOT EXIST!"
        csvfile = urllib2.urlopen("https://dl.dropboxusercontent.com/u/28535341/oot0.csv")
        output = open('oot0.csv','wb')
        output.write(csvfile.read())
        output.close()  
#DOWLOADING FILE FROM DROPBOX FIRST TIME

def psi(bench, comp, group):
    ben_len=len(bench);
    comp_len=len(comp);
    bench.sort();
    comp.sort();
    psi_cut=[];
    n=int(math.floor(ben_len/group));
    for i in range(1,group):
        lowercut=bench[(i-1)*n+1];
        if i!=group:
            uppercut=bench[(i*n)];
            ben_cnt=n;
        else:
            uppercut=bench[-1];
            ben_cnt=ben_len-group*(n-1)
    comp_cnt = len([i for i in comp if i > lowercut and i<=uppercut]);
    ben_pct=(ben_cnt+0.0)/ben_len;
    comp_pct=(comp_cnt+0.0)/comp_len;
    if comp_pct > 0.0:
        psi_cut.append((ben_pct-comp_pct)*math.log(ben_pct/(comp_pct)));
    else:
        psi_cut.append(0);
    psi=sum(psi_cut);
    return psi;
    
print "LOADING DATASETS..."
df = pd.read_csv("dev.csv") #DEV-SAMPLE
dfo = pd.read_csv("oot0.csv")#OUT-OF-TIME SAMPLE

print "IDENTIFYING TYPES..."
in_model = []
list_ib = set()  #input binary
list_icn = set() #input categorical nominal
list_ico = set() #input categorical ordinal
list_if = set()  #input numerical continuos (input float)
list_inputs = set()
output_var = 'ob_target'

for var_name in df.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
        print var_name,"is input"   
    if re.search('^ib_',var_name):
        list_ib.add(var_name)
        print var_name,"is input binary"
    elif re.search('^icn_',var_name):
        list_icn.add(var_name)
        print var_name,"is input categorical nominal"
    elif re.search('^ico_',var_name):
        list_ico.add(var_name)
        print var_name,"is input categorical ordinal"
    elif re.search('^if_',var_name):
        list_if.add(var_name)
        print var_name,"is input numerical continuos (input float)"
    elif re.search('^ob_',var_name):
        output_var = var_name
    else:
        print "ERROR: unable to identify the type of:", var_name


print "STEP 1: DOING MY TRANSFORMATIONS..."
#Será necesario transformar las variables categoricas ordinales, en variables 
print " ------------ PSI for DEV dataset-------------"
import re
import math
list_inputs = set()
var_to_check = set()
for var_name in df.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
for var_name in list_inputs:
    psi_value=psi(bench=list(df[var_name]),comp=list(df[var_name]),group=max(2,min((len(set(df[var_name]))),10)));
    if psi_value > 0.1:
        print "psi for ", var_name, " = ", psi_value
        var_to_check.add(var_name)

print " ------------ PSI for OOT dataset-------------"
list_inputs = set()
var_to_check = set()
for var_name in dfo.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
for var_name in list_inputs:
    psi_value=psi(bench=list(dfo[var_name]),comp=list(dfo[var_name]),group=max(2,min((len(set(dfo[var_name]))),10)));
    if psi_value > 0.1:
        print "psi for ", var_name, " = ", psi_value
        var_to_check.add(var_name)

dfo1 = dfo.fillna(dfo.mean())

print "STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL..."
#in_model = list_inputs #['ib_var_1','icn_var_22','ico_var_25','if_var_65']
#in_model = ['ib_var_1','icn_var_22','ico_var_25','if_var_65']
####-------------------------------------------#####
from sklearn.ensemble import RandomForestClassifier

list_inputs = list(set(list_inputs))
X = df[list_inputs]
y = df[output_var]

print "STEP 2.1: FEATURE IMPORTANCE WITH RANDOM FOREST..."
# Build a forest and compute the feature importances
forest = RandomForestClassifier(n_estimators=250,
                              random_state=0)
forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
# Print the feature ranking
in_model = set()
print("Feature ranking:")
for f in range(81):
    print("%d. feature %s (%f) " % (f + 1, list_inputs[indices[f]], importances[indices[f]] ))
    in_model.add(list_inputs[indices[f]])

in_model = list(in_model) 
print in_model
####-------------------------------------------#####

print "STEP 3: DEVELOPING THE MODEL..."
X = df[list(in_model)]
y = df[output_var]

Xo = dfo1[list(set(in_model))]

#model = sm.Logit(y,X)

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

try:
#     model = sm.GLM(y,X, family = sm.families.Binomial())
#     result = model.fit()
#     print result.summary()
#     y_pred = result.predict(X)
#     yo_pred = result.predict(Xo)

#     model = tree.DecisionTreeClassifier()
#     result = model.fit(X, y)
#     #print result.summary()
#     y_pred = result.predict(X)
#     yo_pred = result.predict(Xo)

    model = RandomForestClassifier(n_estimators=100, criterion='gini')
    result = model.fit(X, y)
    #print result.summary()
    y_pred = result.predict(X)
    yo_pred = result.predict(Xo)
    
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

print "STEP 4: ASSESSING THE MODEL..."
# CALCULATING GINI PERFORMANCE ON DEVELOPMENT SAMPLE
from sklearn.metrics import roc_auc_score
gini_score = 2*roc_auc_score(df[output_var], y_pred)-1
print "GINI DEVELOPMENT=", gini_score

print "STEP 5: SUBMITTING THE RESULTS..."
dfo['pred'] = yo_pred
dfo_tosend = dfo[list(['id','pred'])]

i=1
filename = "group_Z_sub"+str(i)+".csv"
dfo_tosend.to_csv(filename, sep=',')

url = 'http://mgadi.pythonanywhere.com/api/v1.0/uploadpredictions'

files = {'file': (filename, open(filename, 'rb'))}
rsub = requests.post(url, files=files, auth=HTTPBasicAuth('amarino', 'amarino'))
resp_str = str(rsub.text)
print "RESULT SUBMISSION: ", resp_str





IMPORTING LIBRARIES...
LOADING DATASETS...
IDENTIFYING TYPES...
id is input
ERROR: unable to identify the type of: id
ib_var_1 is input
ib_var_1 is input binary
ib_var_2 is input
ib_var_2 is input binary
ib_var_3 is input
ib_var_3 is input binary
ib_var_4 is input
ib_var_4 is input binary
ib_var_5 is input
ib_var_5 is input binary
ib_var_6 is input
ib_var_6 is input binary
ib_var_7 is input
ib_var_7 is input binary
ib_var_8 is input
ib_var_8 is input binary
ib_var_9 is input
ib_var_9 is input binary
ib_var_10 is input
ib_var_10 is input binary
ib_var_11 is input
ib_var_11 is input binary
ib_var_12 is input
ib_var_12 is input binary
ib_var_13 is input
ib_var_13 is input binary
ib_var_14 is input
ib_var_14 is input binary
ib_var_15 is input
ib_var_15 is input binary
ib_var_16 is input
ib_var_16 is input binary
ib_var_17 is input
ib_var_17 is input binary
ib_var_18 is input
ib_var_18 is input binary
ib_var_19 is input
ib_var_19 is input binary
ib_var_20 is input
ib_var_20 is input binary


In [51]:
dfo_tosend['pred'].values().unique()
#dfo.isnull().values

TypeError: 'numpy.ndarray' object is not callable

In [43]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

        
def psi(bench, comp, group):
    ben_len=len(bench);
    comp_len=len(comp);
    bench.sort();
    comp.sort();
    psi_cut=[];
    n=int(math.floor(ben_len/group));
    for i in range(1,group):
        lowercut=bench[(i-1)*n+1];
        if i!=group:
            uppercut=bench[(i*n)];
            ben_cnt=n;
        else:
            uppercut=bench[-1];
            ben_cnt=ben_len-group*(n-1)
    comp_cnt = len([i for i in comp if i > lowercut and i<=uppercut]);
    ben_pct=(ben_cnt+0.0)/ben_len;
    comp_pct=(comp_cnt+0.0)/comp_len;
    if comp_pct > 0.0:
        psi_cut.append((ben_pct-comp_pct)*math.log(ben_pct/(comp_pct)));
    else:
        psi_cut.append(0);
    psi=sum(psi_cut);
    return psi;
    
print "STEP 1: DOING MY TRANSFORMATIONS..."
#Será necesario transformar las variables categoricas ordinales, en variables 
#Utilizamos el PSI para localizar cambios incorrectos en variables así como localizar NaN
print " ------------ PSI for DEV dataset-------------"
import re
import math
list_inputs = set()
var_to_check = set()
for var_name in df.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
for var_name in list_inputs:
    psi_value=psi(bench=list(df[var_name]),comp=list(df[var_name]),group=max(2,min((len(set(df[var_name]))),10)));
    if psi_value > 0.1:
        print "psi for ", var_name, " = ", psi_value
        var_to_check.add(var_name)

print " ------------ PSI for OOT dataset-------------"
list_inputs = set()
var_to_check = set()
for var_name in dfo.columns:
    if re.search('^i',var_name):
        list_inputs.add(var_name)
for var_name in list_inputs:
    psi_value=psi(bench=list(dfo[var_name]),comp=list(dfo[var_name]),group=max(2,min((len(set(dfo[var_name]))),10)));
    if psi_value > 0.1:
        print "psi for ", var_name, " = ", psi_value
        var_to_check.add(var_name)

dfo1 = dfo.fillna(dfo.mean())
        
print "STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL..."
#in_model = list_inputs #['ib_var_1','icn_var_22','ico_var_25','if_var_65']
#in_model = ['ib_var_1','icn_var_22','ico_var_25','if_var_65']
####-------------------------------------------#####
from sklearn.ensemble import ExtraTreesClassifier

list_inputs = list(set(list_inputs))
X = df[list_inputs]
y = df[output_var]

print "STEP 2.1: FEATURE IMPORTANCE WITH RANDOM FOREST..."
# Build a forest and compute the feature importances
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
# Print the feature ranking
in_model = set()
print("Feature ranking:")
for f in range(70):
    print("%d. feature %s (%f) " % (f + 1, list_inputs[indices[f]], importances[indices[f]] ))
    in_model.add(list_inputs[indices[f]])

in_model = list(in_model) 
print in_model
####-------------------------------------------#####

print "STEP 3: DEVELOPING THE MODEL..."
X = df[list(in_model)]
y = df[output_var]

Xo = dfo1[list(set(in_model))]

#model = sm.Logit(y,X)
from sklearn import tree
try:
#     model = sm.GLM(y,X, family = sm.families.Binomial())
#     result = model.fit()
#     print result.summary()
#     y_pred = result.predict(X)
#     yo_pred = result.predict(Xo)
    
    model = tree.DecisionTreeClassifier()
    result = model.fit(X, y)
    #print result.summary()
    y_pred = result.predict(X)
    yo_pred = result.predict(Xo)
#   visualize_tree(result, X.columns)
    
except np.linalg.linalg.LinAlgError as err:
    if 'Singular matrix' in err.message:
        print "MODEL-INVALID (Singular Matrix)"
    else:
        raise

print "STEP 4: ASSESSING THE MODEL..."
# CALCULATING GINI PERFORMANCE ON DEVELOPMENT SAMPLE
from sklearn.metrics import roc_auc_score
gini_score = 2*roc_auc_score(df[output_var], y_pred)-1
print "GINI DEVELOPMENT=", gini_score

STEP 1: DOING MY TRANSFORMATIONS...
 ------------ PSI for DEV dataset-------------
psi for  ib_var_20  =  0.17451251714
psi for  ib_var_19  =  0.275016288497
psi for  ib_var_15  =  0.230828737949
psi for  ib_var_16  =  0.208713388894
psi for  ib_var_17  =  0.267449779291
 ------------ PSI for OOT dataset-------------
STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL...
STEP 2.1: FEATURE IMPORTANCE WITH RANDOM FOREST...
Feature ranking:
1. feature if_var_68 (0.026441) 
2. feature if_var_79 (0.022224) 
3. feature icn_var_24 (0.020946) 
4. feature ico_var_35 (0.020115) 
5. feature if_var_65 (0.019753) 
6. feature if_var_72 (0.019365) 
7. feature if_var_69 (0.019271) 
8. feature id (0.017966) 
9. feature if_var_76 (0.017957) 
10. feature if_var_70 (0.017495) 
11. feature if_var_81 (0.017361) 
12. feature if_var_67 (0.017300) 
13. feature if_var_71 (0.017178) 
14. feature ico_var_34 (0.017042) 
15. feature ico_var_36 (0.016965) 
16. feature ico_var_31 (0.016831) 
17. feature if_var_

In [21]:
df['ib_var_17'].describe()
#df[df['ib_var_20']<1.0]['ib_var_20'].count()
#len(df)

count    864.000000
mean       0.930556
std        0.254356
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: ib_var_17, dtype: float64

In [68]:
df['ib_var_15'].isnull().any()
#df[df['ib_var_20']<1.0]['ib_var_20'].count()
#len(df)

False

In [30]:
%matplotlib inline  
df.head(5)
df['if_var_65'].describe()

count    864.000000
mean      38.409722
std        8.677094
min       21.000000
25%       32.000000
50%       38.000000
75%       44.000000
max       63.000000
Name: if_var_65, dtype: float64

In [31]:
dfo['if_var_65'].describe()

count    2967.000000
mean       38.665993
std         8.629404
min        18.000000
25%        32.000000
50%        38.000000
75%        45.000000
max        81.000000
Name: if_var_65, dtype: float64

In [42]:
dfo.isnull().any().any()
dfo1 = dfo.fillna(dfo.mean())
dfo1.isnull().any().any()

False

In [44]:
print "STEP 5: SUBMITTING THE RESULTS..."
dfo['pred'] = yo_pred
dfo_tosend = dfo[list(['id','pred'])]

STEP 5: SUBMITTING THE RESULTS...


In [45]:
dfo_tosend

,id,pred
0,1,0
1,2,0
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,1
